In [ ]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame, Panel
%pylab inline
#plt.rcParams['figure.figsize'] = (15, 15)

### get soll ###
df_soll = pd.read_csv('~/catkin_ws/src/pitasc/applications/sysident/step_log/2017-11-03/2017-11-03_14-27-47_control_output.log',
                 header=0,
                 names=['time', 'localtime', 'x_soll'])
print df_soll.head()

#remove trailing [
df_soll = df_soll.set_index('localtime')

### get ist ###
df_ist = pd.read_csv('~/catkin_ws/src/pitasc/applications/sysident/step_log/2017-11-03/2017-11-03_14-27-48_task_vel.log',
                 header=0,
                 names=['time', 'localtime', 'x_ist'])

df_ist = df_ist.set_index('localtime')
print df_ist.head()

### make one df with ist and soll; indexed by time
# Concates both series to one and fills (unknown) data with last valid one
df_ist_soll = pd.concat([df_soll.x_soll, df_ist.x_ist], axis=1).fillna(method='pad')
# Fills first value with 0 (there is no valid before that one)
df_ist_soll = df_ist_soll.fillna(0)
df_ist_soll.plot(style='-', drawstyle="steps")

In [ ]:
# Load in the r magic
%load_ext rpy2.ipython

In [ ]:
T = df_ist_soll.index.tolist()
yout = (5 * df_ist_soll['x_ist']).tolist()
xin = (5 * df_ist_soll['x_soll']).tolist()

# Don't know how to import more than one per line...
%R -i T
%R -i yout
%R -i xin

In [ ]:
%%R -o sys
library(sysid)
require(ggplot2)

data_tf <- idframe(output = yout, input = xin)
sys = arx(data_tf, c(1,2,0))
#ls()
#names(sys)
#sys['fitted.values']
plot(T, yout)
lines(T, predict(sys, data_tf, nahead=2))
#print(sys)
names(sys)
print(sys)

In [ ]:
#%R print(sys)

In [ ]:
# Testsystem aufsetzen
import control as con

# pt1 System
k = 0.2
t = 1e-2

# Polynom
tf_pt1 = con.matlab.tf(k, [t, 1])
print tf_pt1


# pt2 System
K = 0.2
d = 0.83
T = .1
delay = 0.246

a0 = 1
a1 = (2 * d * T) #16
a2 = (T**2) #100
b0 = K

# Polynom
tf_pt2 = con.matlab.tf(K, [a2, a1, a0])
print tf_pt2

# Zustandsraum
ss_1a = con.matlab.tf2ss(tf_pt2)
#print ss_1a

# Füge Zeitversatz zu
d_num, d_den = con.pade(delay, 1)
tf_delay = con.tf(d_num, d_den)
ss_pt2_delay = con.series(tf_delay, tf_pt2)

tf_pt1_delay = con.tf(d_num, d_den)
ss_pt1_delay = con.series(tf_delay, tf_pt1)

#print con.matlab.tf2ss(ss_delay)

import matplotlib.pyplot as plt
%pylab inline
d_yout, d_T = con.matlab.step(ss_pt2_delay)
yout, T = con.matlab.step(tf_pt2) # step without delay

#plt.plot(d_T, d_yout, 'r-', label='poly_est')
plt.plot(np.add(d_T, delay), yout, 'r-', label='pt2') #delay in timeaxis!

d_yout, d_T = con.matlab.step(ss_pt1_delay)
yout, T = con.matlab.step(tf_pt1) # step without delay

#plt.plot(d_T, d_yout, 'r-', label='poly_est')
plt.plot(np.add(d_T, delay), yout, 'y-', label='pt1') #delay in timeaxis!


plt.legend()

plt.plot(df_ist_soll)
#df_ist_soll.plot(ylim=[-0.01, 0.21], xlim=[0.246, 1.2-0.246], style='-', drawstyle="steps")